# LLM Evaluation Methods

**Objective**:
Evaluate and compare the performance of two LLMs (*gpt-3.5-turbo* and *gpt-4*) using:

* Automatic evaluation metrics
* Human evaluation
* LLM as a judge

## Step 1: Setup

First, install the necessary libraries and import them.

In [ ]:
!pip install rouge-score bert-score pandas

from rouge_score import rouge_scorer
from bert_score import score
import random
import pandas as pd

## Step 2: Define Prompts

Create a list of prompts to test the models.

In [ ]:
prompts = [
    "Explain the concept of gravity in simple terms.",
    "What are the benefits of regular exercise?",
    "Describe the plot of 'Alice in Wonderland'.",
    "How does a refrigerator work?",
    "Translate 'Good morning' into French."
]

## Step 3: Generate Responses from Two Models

We’ll use two different models: gpt-3.5-turbo and gpt-4.

In [ ]:
# gpt-35-turbo (version:0301)
responses_a = [
    "Gravity is the force that pulls objects towards each other.",
    "Regular exercise has many benefits including improved physical and mental health, better sleep, weight management, increased energy, improved brain function, and social benefits.",
    "Alice's Adventures in Wonderland is a classic novel by Lewis Carroll about a young girl who falls down a rabbit hole into a fantasy world filled with strange and whimsical characters.",
    "A refrigerator works by removing heat from the interior of the refrigerator and releasing it to the surrounding environment through a process of refrigeration.",
    "Bonjour"
]

Generate responses for each prompt.

In [ ]:
# gpt4 (version:0613)
responses_b = responses = [
    "Gravity is a natural force that attracts objects towards each other. It's the reason why when we drop something, it falls to the ground instead of floating in the air. This force also keeps us anchored to the ground, so we don't float off into space. The more massive an object is, the stronger its gravitational pull. So, the Earth, being quite large, has enough gravity to pull everything towards it. In the larger picture, gravity is also what keeps the planets in our solar system orbiting around the sun.",
    "Regular exercise has numerous health benefits, including: 1. Improved Heart Health: Regular exercise strengthens the heart and cardiovascular system, reducing the risk of heart diseases. 2. Weight Control: It helps to manage weight as it burns calories. The more intense the activity, the more calories you burn. 3. Enhanced Mental Health: Physical activity releases chemicals in your brain that act as natural mood lifters. It also helps in reducing stress and anxiety. 4. Improved Sleep: Regular physical activity can help you fall asleep faster and deepen your sleep. 5. Boosted Energy: Regular physical activity improves muscle strength and boosts endurance, giving you more energy. 6. Better Bone Density: Weight-bearing exercises increase bone density, reducing the risk of osteoporosis. 7. Enhanced Immunity: Regular, moderate-intensity exercise can reduce your risk of getting a cold, flu, or other viral illness. 8. Increased Lifespan: People who engage in regular physical activity tend to live longer than those who are inactive. 9. Improved Flexibility and Mobility: Regular exercise keeps your body flexible and helps maintain mobility as you age. Remember, it's important to consult with a healthcare professional before starting a new exercise regime.",
    "\"Alice's Adventures in Wonderland,\" written by Lewis Carroll, is a fantastical tale about a young girl named Alice who falls down a rabbit hole into a bizarre and nonsensical world. The story begins with Alice feeling bored while sitting on the riverbank with her sister. She notices a talking, clothed White Rabbit with a pocket watch run past. Curious, she follows the Rabbit down the rabbit hole into Wonderland. In Wonderland, Alice experiences numerous strange adventures, meeting a variety of eccentric characters. These include the Duchess, the Caterpillar, the Cheshire Cat, the Mad Hatter, the March Hare, the Dormouse, the Queen, and King of Hearts. She grows and shrinks in size several times after consuming certain foods and drinks or using certain magical objects. Alice's encounters are characterized by paradoxes, nonsensical conversations, and absurd characters. One of the most famous events is the Mad Hatter's tea party, where time has stopped, and it's always tea time. The climax occurs with a chaotic trial headed by the Queen of Hearts, who has a penchant for ordering beheadings at the drop of a hat. Alice eventually offends the Queen, who orders her execution. However, Alice grows to a large size and dismisses the Queen's threats, causing the whole pack of cards to fly up into the air and attack her. Alice then wakes up on the riverbank, realizing that her entire adventure in Wonderland was just a dream. Her sister then sends her off for tea, marking the end of the story.",
    "A refrigerator works by removing heat from the items stored inside and releasing it outside, thus cooling the inside of the refrigerator to a temperature below the room temperature. Here's a simplified explanation of the process: 1. The refrigeration cycle starts with the refrigerant (a cooling agent) in the compressor, which is located at the back of the refrigerator. 2. The compressor pressurizes the refrigerant gas, causing it to become hot. 3. This hot gas then travels through the coils on the back or bottom of the refrigerator where it dissipates its heat and cools down, turning into a high-pressure liquid. 4. This high-pressure liquid refrigerant then passes through a small hole to a low-pressure zone. As it does this, it rapidly expands and turns into a cold gas in the process. 5. The cold gas then absorbs heat from inside the refrigerator, cooling down the air and the items inside. 6. The refrigerant, now warmed slightly and turned back into a gas, is drawn back into the compressor, and the cycle starts again. In this way, a refrigerator continually transports heat from inside the box to the outside, keeping your food cool and fresh.",
    "\"Good morning\" in French is \"Bonjour\"."
]

## Step 4: Human Evaluation



**Now it’s *your* turn to evaluate!**

Before diving into automated evaluation metrics, let’s see how you, as a human evaluator, would rate the responses.

Instructions:

1.	For each prompt, read both responses (Response 1 and Response 2).
2.	Without knowing which model generated the responses, rate each response on a scale of 1 (poor) to 5 (excellent) based on quality, relevance, and completeness.
3.	Record your ratings.

Presenting the responses for evaluation:

In [ ]:
for i, prompt in enumerate(prompts):
    print(f"Prompt {i+1}: {prompt}\n")
    print(f"Response 1:\n{responses_a[i]}\n")
    print(f"Response 2:\n{responses_b[i]}\n")
    print("-" * 80)

### Please record your ratings in the following lists:

In [ ]:
human_ratings_response_a = [None, None, None, None, None]
human_ratings_response_b = [None, None, None, None, None]


df_human_a = pd.DataFrame(human_ratings_response_a)
df_human_a.index = [f"Prompt {i+1}" for i in range(len(prompts))]
df_human_a.rename(columns=lambda x: "Human_eval", inplace=True)

df_human_b = pd.DataFrame(human_ratings_response_b)
df_human_b.index = [f"Prompt {i+1}" for i in range(len(prompts))]
df_human_b.rename(columns=lambda x: "Human_eval", inplace=True)

df_human_a.rename(columns=lambda x: f"Model_A_{x}", inplace=True)
df_human_b.rename(columns=lambda x: f"Model_B_{x}", inplace=True)

# merge outputs
df_human = pd.concat([df_human_a, df_human_b], axis=1)
df_human


df_human_eval = pd.DataFrame(df_human, index=[f"Prompt {i+1}" for i in range(len(prompts))])
df_human_eval

## Step 5: Automatic Evaluation Using ROUGE Scores

Use ROUGE scores to evaluate the similarity between the responses.

In [ ]:
reference_answers = [
    "Gravity is a force that attracts objects with mass towards each other. It's what keeps us grounded on Earth and makes things fall when you drop them.",
    "Regular exercise improves cardiovascular health, strengthens muscles, enhances flexibility, aids in weight management, and boosts mental well-being by reducing stress and anxiety.",
    "In 'Alice in Wonderland', a young girl named Alice falls through a rabbit hole into a fantastical world filled with peculiar creatures and bizarre adventures, challenging her perception of reality.",
    "A refrigerator works by using a refrigerant to absorb heat from the interior and release it outside. This process involves compression, condensation, expansion, and evaporation to keep the inside cool.",
    "Bonjour"
]

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def compute_rouge_scores(refs, hyps):
    scores = []
    for ref, hyp in zip(refs, hyps):
        score = scorer.score(ref, hyp)
        scores.append({
            'rouge1_fmeasure': score['rouge1'].fmeasure,
            'rougeL_fmeasure': score['rougeL'].fmeasure
        })
    return scores

Compute ROUGE scores between the two sets of responses.

In [ ]:
rouge_scores = compute_rouge_scores(responses_a, reference_answers)
df_rouge_a = pd.DataFrame(rouge_scores)
df_rouge_a.index = [f"Prompt {i+1}" for i in range(len(prompts))]
df_rouge_a

In [ ]:
rouge_scores = compute_rouge_scores(responses_b, reference_answers)
df_rouge_b = pd.DataFrame(rouge_scores)
df_rouge_b.index = [f"Prompt {i+1}" for i in range(len(prompts))]
df_rouge_b

In [ ]:
df_rouge_a.rename(columns=lambda x: f"Model_A_{x}", inplace=True)
df_rouge_b.rename(columns=lambda x: f"Model_B_{x}", inplace=True)

# merge outputs
df_rouge = pd.concat([df_rouge_a, df_rouge_b], axis=1)
df_rouge

Computing BERTScore for each model:

In [ ]:
def compute_bertscore(references, candidates):
    P, R, F1 = score(candidates, references, lang='en', verbose=True)
    return F1.tolist()

In [ ]:
# Model A
bertscores_a = compute_bertscore(reference_answers, responses_a)
df_bertscore_a = pd.DataFrame({
    'BERTScore(F1)': bertscores_a
}, index=[f"Prompt {i+1}" for i in range(len(prompts))])
df_bertscore_a.rename(columns=lambda x: f"Model_A_{x}", inplace=True)

# Model B
bertscores_b = compute_bertscore(reference_answers, responses_b)
df_bertscore_b = pd.DataFrame({
    'BERTScore (F1)': bertscores_b
}, index=[f"Prompt {i+1}" for i in range(len(prompts))])
df_bertscore_b.rename(columns=lambda x: f"Model_B_{x}", inplace=True)

# merge outputs
df_bertscore = pd.concat([df_bertscore_a, df_bertscore_b], axis=1)
df_bertscore

## Step 6: LLM as a Judge

Use an LLM to judge which response is better for each prompt.

- used gpt-4 to evaluate

In [ ]:
def llm_judge(prompt, response_a, response_b):
    judge_prompt = f"""
You are an expert evaluator.

Prompt: {prompt}

Response A: {response_a}

Response B: {response_b}

Question: Answer shortly which response is better, Response A or Response B? Provide a brief justification.
"""
    print(judge_prompt)

Evaluate each pair of responses.

In [ ]:
judgements = []
for i in range(len(prompts)):
    judgement = llm_judge(prompts[i], responses_a[i], responses_b[i])
    judgements.append(judgement)

Create a DataFrame of the judgements.

In [ ]:
gpt4_judgements = ["Response B is better because it provides a more detailed and complete explanation of the concept of gravity, including examples and how it affects our daily lives. It also mentions its role in the solar system, which helps to broaden the understanding of the topic.",
 "Response B is better as it provides more specific and detailed benefits of regular exercise, backed by scientific evidence. It also includes a reminder to consult with a healthcare professional before starting a new exercise regime, which is important for safety.",
"Response B is better because it provides a more detailed and complete plot summary of \"Alice in Wonderland.\" It includes important characters, events, and themes of the story, making it more informative and engaging for the reader.",
 "Response B is better. It provides a more detailed and step-by-step explanation of how a refrigerator works. This response not only gives a general overview but also explains the refrigeration cycle in a clear and easy-to-understand manner. It's more informative and helpful for someone looking to understand the process thoroughly.",
 "Response B is better. While both responses are correct, Response B provides a complete sentence, making it clearer for the user. It reiterates the original prompt (translating \"Good morning\" into French) and then provides the answer, which can be particularly helpful for clarity in a written or digital communication context."]

df_judgements = pd.DataFrame(gpt4_judgements, index=[f"Prompt {i+1}" for i in range(len(prompts))], columns=["llm as a judge"])

## Step 7: Analysis

Compile all results into a single DataFrame.

In [ ]:
df_results = pd.concat([df_rouge, df_bertscore, df_judgements, df_human_eval], axis=1)
df_results